In [3]:
import os
import sqlite3
from pathlib import Path

In [4]:
# Creating a database:
# If the database you try to connect to does not exist, it will be created
connection = sqlite3.connect('test_database.db')

In [9]:
# execute a structured language query (SQL) to make a table
if not Path('test_database.db'):
    connection.execute("create table customers (first_name text, last_name text, address text, city text, state text, zip text)")

# commit the changes or nothing will be saved
connection.commit()

# close the connection to database
connection.close()

In [10]:
# viewing tables in a database
connection = sqlite3.connect('test_database.db')

# using 'sqlite_master' as a table name in a query will return the available tables
table_names = connection.execute('select name from sqlite_master')
for name in table_names:
    print(name)

# close the db when done. It's important yo
connection.close()

('customers',)


In [3]:
# inserting data into a table
connection = sqlite3.connect('test_database.db')

connection.execute("insert into customers values (?,?,?,?,?,?)",('Charles2', 'Parmley', 'address', 'KC', 'MO', '64105'))
connection.commit()
connection.close()

In [6]:
# Adding multiple values at once
connection = sqlite3.connect('test_database.db')
cur = connection.cursor()
customers = [
    ('Bill', 'Kay', 'address', 'Overland Park', 'MO', '64105'),
    ('Steve', 'Johnson', 'address', 'Lenexa', 'MO', '64105'),
    ('Mark', 'Ida', 'address', 'Blue Valley', 'MO', '64105'),
    ('Roger', 'Bragent', 'address', 'Olathe', 'MO', '64105')
]
for customer in customers:
    cur.execute("insert into customers values (?,?,?,?,?,?)", customer)
connection.commit()
connection.close()

In [19]:
# Reading from multiple rows at once
connection = sqlite3.connect('test_database.db')
cursor = connection.cursor()
customers = connection.execute('select * from customers')

cities = []
for customer in customers:
    cities.append(customer[3])
print(cities)

['KC', 'Overland Park', 'Lenexa', 'Blue Valley', 'Olathe']


In [21]:
# Using core python put cities in alphabetical order
sorted_cities = sorted(cities)
print(sorted_cities)

#to reverse it
sorted_cities.reverse()
print(sorted_cities)

['Blue Valley', 'KC', 'Lenexa', 'Olathe', 'Overland Park']
['Overland Park', 'Olathe', 'Lenexa', 'KC', 'Blue Valley']


In [14]:
# Do the same in SQL using 'order by'
connection = sqlite3.connect('test_database.db')
cursor = connection.cursor()
sorted_customers_by_city = cursor.execute('select * from customers order by city') # add 'desc' at the end for descending order

sorted_cities  = []
for customer in sorted_customers_by_city:
    sorted_cities.append(customer[3])
print(sorted_cities)

['Blue Valley', 'KC', 'Lenexa', 'Olathe', 'Overland Park']


In [27]:
# List all customers whos customers start with a certain letter
connection = sqlite3.connect('test_database.db')
cursor = connection.cursor()
customers = cursor.execute('select * from customers')

count = 0
letter = 'C'
for customer in customers:
    print(customer[0])
    if customer[0][0] == letter:
        count += 1
print(f'{letter} : {count}')

Charles
Bill
Steve
Mark
Roger
1


In [46]:
# Return a count of all customer names with the number of times the first name occurs
connection = sqlite3.connect('test_database.db')
cursor = connection.cursor()
customers = cursor.execute('select * from customers')

# Change the cursor object into a list
customers = list(customers)


# Store customers first names
names = [customer[0] for customer in customers]
name_dict ={name : names.count(name) for name in names}


print(name_dict)


{'Charles': 1, 'Bill': 1, 'Steve': 1, 'Mark': 1, 'Roger': 1}


In [48]:
# fetchall()
connection = sqlite3.connect('test_database.db')
cursor = connection.cursor()

# Grab the table items you want
cursor.execute('select * from customers')

# use .fetchall() on the cursor object to store them as a list of tuples
customers = cursor.fetchall()
print(customers)

[('Charles', 'Parmley', 'address', 'KC', 'MO', '64105'), ('Bill', 'Kay', 'address', 'Overland Park', 'MO', '64105'), ('Steve', 'Johnson', 'address', 'Lenexa', 'MO', '64105'), ('Mark', 'Ida', 'address', 'Blue Valley', 'MO', '64105'), ('Roger', 'Bragent', 'address', 'Olathe', 'MO', '64105')]


In [51]:
# fetchmany()
connection = sqlite3.connect('test_database.db')
cursor = connection.cursor()

cursor.execute('select * from customers')

amount_of_customers_specified = cursor.fetchmany(2)

print(amount_of_customers_specified)

[('Charles', 'Parmley', 'address', 'KC', 'MO', '64105'), ('Bill', 'Kay', 'address', 'Overland Park', 'MO', '64105')]


In [83]:
# using the 'where' clause to fetchone()
connection = sqlite3.connect('test_database.db')
cursor = connection.cursor()

name = 'Charles'
city = 'KC'
specified_customers = cursor.execute("select * from customers where first_name = 'Charles'")

customers = [customer for customer in specified_customers]
print(customers)
connection.close()

[('Charles', 'Parmley', 'address', 'KC', 'MO', '64105')]


In [84]:
# Createa a table called departments
'''

Table structure:
department_code text, department_name text

'''

connection = sqlite3.connect('test_database.db')
cursor.execute('create table departments (department_code text, department_name text)')
connection.commit()
connection.close()

In [105]:
# fill department table with data
departments = [
('FIN', 'Finance'),
('ACC', 'Accounting'),
('HR', 'Human Resources'),
('I.T.', 'Information Technology'),
('BACK', 'Back Office'),
('SALE', 'Sales'),
]

connection = sqlite3.connect('test_database.db')

# list comprehension to add values ----- slow vs executemany() I think?
# [connection.execute('insert into departments values(?,?)',(department[0],department[1])) for department in departments]

# The better practice is to use .executemany() to insert multiple values
connection.executemany('insert into departments values(?,?)', departments)
connection.commit()
connection.close()

In [110]:
# List all department codes with a inside it using python to filter
connection = sqlite3.connect('test_database.db')
cursor = connection.cursor()

department_data = cursor.execute('select * from departments')
print([department for department in department_data if 'A' in department[0]])
connection.close()


[('ACC', 'Accounting'), ('BACK', 'Back Office'), ('SALE', 'Sales')]


In [134]:
# List all department codes with a inside it using SQL syntax to filter

'''
LIKE ‘a%’ - Searches for all the values starting with a.

LIKE ‘%a’  -Searches for all the values which end with a.

LIKE ‘_a%’ - Searches for all the values where ‘a’ is at the second position. 
    -The '_' represents a single character, the second character must be ‘a’ and 
     after 'a' there might be some characters present or not.
'''


connection = sqlite3.connect('test_database.db')
cursor = connection.cursor()
items = cursor.execute("select department_code from departments where department_code not like 'A'")

[print(item) for item in items]
connection.close()

('FIN',)
('ACC',)
('HR',)
('I.T.',)
('BACK',)
('SALE',)


In [135]:
# Homework: 
    # Iterate department names and return:
        # count of each letter regardless of case
        # count of blank spaces
        # list of letters found in alphabetic order

